<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/6%20-%20Sentiment%20Analysis/classifier_nltk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creare un classificatore con NLTK
In questo notebook vedremo come creare un classificatore usando NLTK per eseguire la sentimet analisys sull'IMDB Movies Review Dataset, in modo tale da classificare recensioni positive e negative. Il modello che creeremò sarà un classificatore bayesiano, che si presta molto bene a problemi di classificazione di documenti di testo.
<br><br>
Cominciamo importando nltk e, se non lo abbiamo già fatto, scarichiamo i moduli per le stopwords e la creazione dei tokens.

In [28]:
import nltk

nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

## Procuriamoci il dataset
Cominciamo scaricando il dataset, esegui la cella di codice qui sotto se sei su Google Colab o se hai wget installato sul tuo computer, altrimenti puoi scaricare il dataset da questo [link](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz).


In [0]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

L'archivio è compresso in formato tar.gz, esegui la cella di codice qui sotto se sei su Colab o hai l'utility tar installata sul tuo PC (solitamente è installata di base su Linux/OsX) se sei su Windows puoi usare 7-zip (può essere che anche winrar vada bene per estrarlo).

In [0]:
!tar -xzf aclImdb_v1.tar.gz

Se abbiamo estratto il dataset adesso avremo una cartella aclImdb con dentro diversi file e altre due cartelle, una con le recensioni per l'addestramento e una con le recensioni per il test. Il dataset ci fornisce il testo di ogni recensione, ognuna in un file txt differente, organizzate in cartelle corrispondenti al sentiment positivo/negativo. 

## Estrazione delle features
Il classificatore bayesiano di NLTK richiede che gli esempi gli vengano forniti all'interno di un array i cui valori sono un dizionario con dentro la frase e il label. La frase deve a sua volta essere codificata in un dizionario con ogni parola come chiave e il valore True come valore (non chiedermi perché).
<br>
Realizziamo una funzione che ci permetta di creare questo dataset.

In [0]:
from os import listdir
from nltk.corpus import stopwords
import string


def get_dataset(files_path, labels=["pos","neg"], samples_per_class=None):
     
    dataset = []
        
    for label in labels:
      count = 0
      path = files_path+label
      for file in listdir(path):
        review_file = open(path+"/"+file)
        review = review_file.read()    
        review = review.translate(string.punctuation)
        words = nltk.word_tokenize(review)
        
        #words_filtered = [word for word in words if word not in stopwords.words("english")]
        #words_dict = dict([(word, True) for word in words_filtered])      
  
        words_dict = dict([(word, True) for word in words if word not in stopwords.words("english")])
  
        dataset.append((words_dict,label))
                  
        count+=1
        
        if(samples_per_class!=None):
          if(count>=samples_per_class):
            break
    
    return dataset

Nella funzione abbiamo definito un parametro che ci permette di controllare gli esempi massimi per classe, infatti se cerchiamo di caricare tutte le 50.000 recensioni tra set di addestramento e di test, questo richiederebbe molto tempo, limitiamoci a 1000 per il momento per il set di addestramento e 500 per il set di test.

In [60]:
train_set = get_dataset("aclImdb/train/", samples_per_class=1000)

print("Prima recensione del train set")
print(train_set[0][0])
print("Sentiment: %s" % train_set[0][1])

Prima recensione del train set
{'Just': True, 'finished': True, 'watching': True, 'movie': True, 'wanted': True, 'give': True, 'opinion': True, '(': True, 'justice': True, ')': True, 'movie.': True, '<': True, 'br': True, '/': True, '>': True, 'First': True, ',': True, 'get': True, 'things': True, 'straight': True, 'pretending': True, 'anything': True, 'solid': True, 'action': True, 'comedy': True, '.': True, 'It': True, "n't": True, 'aim': True, 'revolutionize': True, 'industry': True, 'garner': True, 'critical': True, 'acclaims': True, 'want': True, 'regarded': True, 'one': True, 'If': True, 'really': True, 'enjoy': True, 'fullest': True, 'I': True, 'suggest': True, 'discard': True, 'critical-mindedness': True, 'longing': True, 'good': True, 'plot': True, 'wo': True, 'find': True, 'With': True, 'established': True, 'let': True, 'us': True, 'low': True, 'expectations': True, 'simply': True, 'strong': True, 'Yes': True, 'moviegoers': True, 'underrated': True, 'well': True, 'never': Tru

In [61]:
test_set = get_dataset("aclImdb/test/", samples_per_class=500)

print("Prima recensione del test set")
print(test_set[0][0])
print("Sentiment: %s" % test_set[0][1])

Prima recensione del test set
{'Although': True, 'little': True, 'pleasant': True, '11-minute': True, 'musical': True, 'diversion': True, '(': True, "'s": True, 'rightly': True, 'billed': True, '``': True, 'Tabloid': True, 'Musical': True, "''": True, ')': True, 'EVERY': True, 'Sunday': True, 'one': True, 'famous': True, 'precious': True, 'documents': True, 'cinematic': True, 'history': True, ',': True, 'since': True, 'provides': True, 'invaluable': True, 'look': True, 'burgeoning': True, 'talents': True, 'two': True, 'screen': True, 'talented': True, 'beloved': True, 'performers': True, ':': True, 'Deanna': True, 'Durbin': True, 'Judy': True, 'Garland.': True, '<': True, 'br': True, '/': True, '>': True, 'often': True, 'cited': True, 'test': True, 'sorts': True, 'produced': True, 'MGM': True, 'adolescent': True, 'appeal': True, 'studio': True, 'contractees': True, 'Garland': True, 'whose': True, 'options': True, 'reportedly': True, 'coming': True, 'renewal': True, 'assertion': True, '

Adesso possiamo addestrare il nostro classificatore utilizzando la funzione *train(train_set)* del modulo *NaiveBayesClassifier* di NLTK.

In [0]:
from nltk.classify import NaiveBayesClassifier

classifier = NaiveBayesClassifier.train(train_set)

Utilizziamo la funzione *accuracy* di NLTK per calcolare l'accuratezza delle predizioni del modello sul set di addestramento e sul set di test.

In [63]:
from nltk.classify.util import accuracy

accuracy_train = nltk.classify.util.accuracy(classifier, train_set)
accuracy_test = nltk.classify.util.accuracy(classifier, test_set)

print("Accuracy sul set di addestramento: %.3f " % accuracy_train)
print("Accuracy sul set di test: %.3f " % accuracy_test)

Accuracy sul set di addestramento: 0.992 
Accuracy sul set di test: 0.792 


Il modello soffre di overfitting ed è normale dato che abbiamo usato soltanto pochi esempi, ma va bene così.<br>  L'oggetto *classifier* ritornato dalla funzione *train* ci permette di ottenere le parole più informative, cioè quelle che se trovate all'interno di una recensione, aumentano di molto la probabilità che questa appartenga all'una o all'altra classe.


In [51]:
classifier.show_most_informative_features()

Most Informative Features
                   waste = True              neg : pos    =     14.6 : 1.0
                 wasting = True              neg : pos    =     13.7 : 1.0
                  wasted = True              neg : pos    =     11.9 : 1.0
                terrific = True              pos : neg    =     11.8 : 1.0
                  deeply = True              pos : neg    =     11.7 : 1.0
                   Davis = True              pos : neg    =     11.7 : 1.0
              underrated = True              pos : neg    =     11.7 : 1.0
                    8/10 = True              pos : neg    =     11.0 : 1.0
              uninspired = True              neg : pos    =     10.3 : 1.0
               stupidity = True              neg : pos    =     10.3 : 1.0


Fatta eccezione per Davis (che non ho idea di chi sia), le parole sono tutte forti indicatori di una recensione negativa (es. waste, unispired, stupidity) o positiva (deeply, terrific, 8/10).
<br>
Probiamo ora ad utilizzare il modello per classificare recensioni scritte da noi, definiamo una funzione che crea il dizionario parola:True partendo dalla recensione

In [0]:
def create_word_features(review):
  review = review.translate(string.punctuation)
  words = nltk.word_tokenize(review)
  words_dict = dict([(word, True) for word in words if word not in stopwords.words("english")])
  return words_dict

Ora utilizziamo il metodo *classify* insieme alla funzione definita sopra per classificare le recensioni.

In [74]:
review = "This movie was just great"
classifier.classify(create_word_features(review))

'pos'

In [75]:
review = "This movie was just terrible"
classifier.classify(create_word_features(review))

'neg'